  Copyright(c) 2021 UNIADEX, Ltd. All Rights Reserved.  
  CONFIDENTIAL  
  Author: UNIADEX, Ltd.

# ショット切り出し  

以下の処理を行います。  

* ショット切り出し
* 中断区間のデータ除外
* 物理変換 + 校正
* SPM計算
* 事象記録のタグ付け
* SPMから算出される、ショット当たりの最大サンプル数を超えたショット除外
* Elasticsearchインデックスへの保存
  * shots-yyyyMMddHHMMSS-data：切り出されたショットデータ
  * shots-yyyyMMddHHMMSS-meta：ショットのメタデータ
  

In [1]:
# ライブラリインポート
import os, sys
sys.path.append(os.path.join(os.getcwd(), '../'))
from backend.cut_out_shot.cut_out_shot import CutOutShot
from backend.common import common
from backend.common.common_logger import logger

SET log configuration (script path): /home/ymiyamoto5/data-analysis-platform/backend/common/logging.conf


### CutOutShot クラスパラメーター一覧  

**【必須】**  
***displacement_func***  
変位センサーの物理変換式  

***load01_func***  
荷重センサー01の物理変換式  

***load02_func***  
荷重センサー02の物理変換式  

***load03_func***  
荷重センサー03の物理変換式  

***load04_func***  
荷重センサー04の物理変換式  


**【任意】**  
***min_spm***  
最小SPM。この値をもとに、1ショットに含まれる最大サンプル数を計算し、最大サンプル数を超えたショットは切り出し対象外とする。  
指定しない場合、既定値の15となる。  

***back_seconds_for_tagging***  
データ収集マネージャーで記録した事象について、何秒遡ってタグ付けするかを指定。指定しない場合、既定値の120となる。

***previous_size***  
ショットを検知したサンプルから何サンプル遡ってショットに含めるかを指定。これは切り出しショットに荷重開始点が確実に含まれるようにするため。指定しない場合、既定値の1000となる。


### cut_out_shot 関数パラメーター一覧
**【必須】**  
***rawdata_dir_name***  
ショット切り出しの対象とする生データファイルのディレクトリ（.pklファイルが含まれるディレクトリ）。  
ディレクトリ名はデータ収集開始日時で、yyyyMMddHHMMSSの形式となっている。

***start_displacement***  
ショット切り出しの開始を検知するための変位値。  

***end_displacement***  
ショット切り出しの終了を検知するための変位値。  

**【任意】**  
***start_sequential_number***  
ショット切り出しの開始連番。指定しない場合、データの先頭から切り出し。  

***end_sequential_number***  
ショット切り出しの終了連番。指定しない場合、データの末尾まで切り出し。  

In [4]:
# データディレクトリの一覧
!ls /mnt/datadrive/data

20210709190000
20210813144147
20210813160141
conf_GW-01.cnf
machine-01-20210101000000
machine-01-20210327141514
machine-01-20210813144147
machine-02_GW-01_AD-USB-1608_20210813-070200.339398.dat
nodes
press_senario.npy


In [5]:
# 物理変換式設定
displacement_func = lambda v: 70.0 - (v - 2.0) * 70.0 / 8.0
Vr = 2.5
load01_func = lambda v: 2.5 / Vr * v
load02_func = lambda v: 2.5 / Vr * v
load03_func = lambda v: 2.5 / Vr * v
load04_func = lambda v: 2.5 / Vr * v

In [6]:
# 適宜変更してください
machine_id = "machine-01"
target_datetime = "20210327141514"
rawdata_dir_name = machine_id + "-" + target_datetime
start_displacement = 47.0
end_displacement = 34.0
# start_displacement=158.0
# end_displacement=-5.0
# start_sequential_number=20000
# end_sequential_number=140000

cut_out_shot = CutOutShot(
    machine_id=machine_id,
    displacement_func=displacement_func,
    load01_func=load01_func,
    load02_func=load02_func,
    load03_func=load03_func,
    load04_func=load04_func,
    margin=0.3,
)

cut_out_shot.cut_out_shot(
    rawdata_dir_name=rawdata_dir_name, 
    start_displacement=start_displacement, 
    end_displacement=end_displacement,
#     start_sequential_number=start_sequential_number,
#     end_sequential_number=end_sequential_number
)

[2021/08/17 14:40:53.247] INFO - cut_out_shot.py#cut_out_shot:450: Cut out shot start.
[2021/08/17 14:40:53.387] INFO - elastic_manager.py#delete_exists_index:164: delete index 'shots-machine-01-20210327141514-data' finished. result: {'acknowledged': True}


/home/ymiyamoto5/data-analysis-platform/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


[2021/08/17 14:40:53.864] INFO - elastic_manager.py#create_index:193: create index 'shots-machine-01-20210327141514-data' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'shots-machine-01-20210327141514-data'}
[2021/08/17 14:40:54.051] INFO - elastic_manager.py#delete_exists_index:164: delete index 'shots-machine-01-20210327141514-meta' finished. result: {'acknowledged': True}
[2021/08/17 14:40:54.474] INFO - elastic_manager.py#create_index:193: create index 'shots-machine-01-20210327141514-meta' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'shots-machine-01-20210327141514-meta'}
[2021/08/17 14:40:54.476] INFO - event_manager.py#fetch_events:74: Fetch events from events-machine-01-20210327141514.
[2021/08/17 14:40:54.821] INFO - cut_out_shot.py#cut_out_shot:600: 7720 data cut out from /mnt/datadrive/data/machine-01-20210327141514/machine-01_GW-01_AD-USB-1608_20210327-062001.118528.pkl.


/home/ymiyamoto5/data-analysis-platform/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


[2021/08/17 14:40:55.280] INFO - throughput_counter.py#throughput_counter:24: Thoughput: 196940 doc/sec, processed_count: 157696
[2021/08/17 14:40:55.429] INFO - cut_out_shot.py#cut_out_shot:600: 7723 data cut out from /mnt/datadrive/data/machine-01-20210327141514/machine-01_GW-01_AD-USB-1608_20210327-062002.688044.pkl.


...

/home/ymiyamoto5/data-analysis-platform/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


[2021/08/17 14:41:05.450] INFO - cut_out_shot.py#cut_out_shot:600: 7733 data cut out from /mnt/datadrive/data/machine-01-20210327141514/machine-01_GW-01_AD-USB-1608_20210327-062021.545709.pkl.


/home/ymiyamoto5/data-analysis-platform/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


[2021/08/17 14:41:06.587] INFO - cut_out_shot.py#cut_out_shot:624: Cut out shot finished.


/home/ymiyamoto5/data-analysis-platform/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
